# 1. Package Installation

In [1]:
!nvidia-smi

Wed Jul 24 03:34:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
#@title Requirements
%%writefile requirements.txt
peft
fire
accelerator
transformers
datasets
evaluate
pyarrow
galore-torch
pytorch-ignite
rouge-score
nltk
py7zr
optimum[exporters]
trl
lightning
jsonargparse[signatures]
deepspeed
colossalai
wandb
tensorrt
nvidia-modelopt --index https://pypi.nvidia.com

Writing requirements.txt


In [3]:
#@title Install Packages
%%capture
!CUDA_EXT=1 DS_BUILD=1 pip install --no-cache -r requirements.txt

In [5]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
!huggingface-cli login --add-to-git-credential



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [6]:
#@title Weight and Bias Train Logger Login
#@markdown weight and bias 로그인
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# 2. Load Model


In [29]:
#@title Get peft model from huggingface
#@markdown Colab 고용량 Ram CPU에서 가능한 범위 ~8B(테스트 중)
#@markdown
#@markdown  |모델       | Normal   | DeepSpeed |
#@markdown  |---        | ---      | ---       |
#@markdown  |Llama3-8B  |  X       |   O       |
#@markdown  |Mistral-7B |  X       |   O       |
#@markdown  |Llama3-70B |  X       |   X       |

%%writefile peft_model.py

import os
import fire
import torch
from peft import AutoPeftModelForCausalLM, LoraConfig
from peft import (inject_adapter_in_model, prepare_model_for_kbit_training,
                  get_peft_model, replace_lora_weights_loftq)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers.utils import PaddingStrategy
from transformers.tokenization_utils_base import TruncationStrategy
from datasets import load_dataset
from random import randint


base_model_id = "Qwen/Qwen2-1.5B-Instruct" # @param ["Gunulhona/tb_pretrained_sts", "Gunulhona/tb_pretrained", "google/flan-t5-xxl", "meta-llama/Meta-Llama-3-8B", "meta-llama/Meta-Llama-3-70B-Instruct", "mistralai/Mistral-7B-Instruct-v0.3", "Qwen/Qwen2-7B-Instruct", "google/gemma-7b", "MLP-KTLim/llama-3-Korean-Bllossom-8B", "EleutherAI/polyglot-ko-12.8b", "vilm/vulture-40b", "arcee-ai/Arcee-Spark", "Qwen/Qwen2-1.5B-Instruct", "OuteAI/Lite-Mistral-150M", "google/gemma-2b-it"] {allow-input: true}

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


peft_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    trust_remote_code=True,
    # quantization_config=bnb_config
    )

peft_model = prepare_model_for_kbit_training(peft_model)

# adapter configuration
lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    init_lora_weights="gaussian", #"gaussian", "pissa", "pissa_niter_{n}", "loftq", False
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    inference_mode=False,
    use_dora=False,
    task_type="CAUSAL_LM",
)

# peft_model.add_adapter(lora_config, adapter_name="adapter_1")
inject_adapter_in_model(lora_config, peft_model, "adapter_1")
peft_model = get_peft_model(peft_model, lora_config)


tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_special_tokens=True,
    trust_remote_code=True)
tokenizer.model_input_names=['input_ids', 'attention_mask']
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"
tokenizer.truncation_side = "right"



Overwriting peft_model.py


#3. Load Dataset

In [8]:
#@title Load data From huggingface datasets
#@markdown summary task에 대해 우선적으로 실험
%%writefile finetuning_datasets.py
import numpy as np
from datasets import load_dataset, concatenate_datasets
from transformers import DataCollatorForSeq2Seq, DataCollatorWithPadding, DataCollatorForLanguageModeling

from evaluate import load
from peft_model import tokenizer
from finetuning_datafunctions import formatting, preprocess_function, SumDataCallator


dataset_path = "Samsung/samsum" # @param ["Samsung/samsum", "emozilla/soda_synthetic_dialogue", "frcp/summary-alpaca-v01"] {allow-input: true}

raw_dataset = load_dataset(
  dataset_path,
  trust_remote_code=True,
  revision="main",  # tag name, or branch name, or commit hash
)

metric = load("rouge")
full_dataset = concatenate_datasets([raw_dataset["train"], raw_dataset["test"]])
tokenized_inputs = full_dataset.map(
    lambda x: tokenizer(x["dialogue"], truncation=True),
    batched=True,
    remove_columns=["dialogue", "summary"])

input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# take 85 percentile of max length for better utilization
max_source_length = int(np.percentile(input_lenghts, 100)) + int(np.percentile(input_lenghts, 10))
max_source_length = min(4096, max_source_length)

tokenized_targets = full_dataset.map(
    lambda x: tokenizer(x["summary"], truncation=True),
    batched=True,
    remove_columns=["dialogue", "summary"])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 90 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 100)) + int(np.percentile(target_lenghts, 10))
max_target_length = min(4096, max_target_length)

dataset = raw_dataset.map(preprocess_function,
                          batched=True,
                          remove_columns=["dialogue", "summary", "id"],
                          fn_kwargs={
                              "max_source_length": max_source_length,
                              "max_target_length": max_target_length
                             },)
# dataset = raw_dataset
# if any([d for d in dataset.values() if "token_type_ids" in d.features]):
#     dataset = dataset.map(lambda x: x,
#                           batched=True,
#                           remove_columns=["token_type_ids"], )


Writing finetuning_datasets.py


In [34]:
#@title Data Modules
#@markdown Data Module Functions
%%writefile finetuning_datafunctions.py
import numpy as np
import torch
from datasets import load_dataset, concatenate_datasets
from transformers import DataCollatorForSeq2Seq, DataCollatorWithPadding, DataCollatorForLanguageModeling

from evaluate import load
from peft_model import tokenizer


def formatting(sample,
               max_source_length,
               max_target_length,
               padding="max_length"):
    # add prefix to the input for t5
    model_inputs, labels = [], []
    for dialogue, summary in zip(sample["dialogue"], sample["summary"]):
        chat_template = [
            # {
            #     "role": "system",
            #     "content": "You are a friendly chatbot who always responds with summary",
            # },
            {
                "role": "user",
                "content": f"Summarize the following dialogue\n\n{dialogue}"
            },

        ]
        label_template = [{
                "role": "assistant",
                "content": f"{summary}"
        }]

        chat_message = tokenizer.apply_chat_template(conversation=chat_template,
                                                     tokenize=False,
                                                     add_generateion_prompt=False, )
        bot_message = tokenizer.apply_chat_template(conversation=label_template,
                                                    tokenize=False,
                                                    add_generateion_prompt=False, )
        model_inputs.append(chat_message)
        labels.append(bot_message)

    # inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    # model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True, )

    # Tokenize targets with the `text_target` keyword argument
    # labels = tokenizer(text_target=sample["summary"],
    #                    max_length=max_target_length,
    #                    padding=padding,
    #                    truncation=True,)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    # if padding == "max_length":
    #     labels["input_ids"] = [
    #         [(l if l != tokenizer.pad_token_id else 1) for l in label] for label in labels["input_ids"]
    #     ]
    # model_inputs["labels"] = labels["input_ids"]
    return model_inputs, labels

def preprocess_function(sample, max_source_length, max_target_length):
    templated_text, labels = formatting(sample, max_source_length, max_target_length)

    return {
        "input_ids": templated_text,
        "labels": labels
    }

class CallatorOutput:
    def __init__(self, input_ids, attention_mask, labels):
        self._input_ids = input_ids
        self._attention_mask = attention_mask
        self._labels = labels

    def __len__(self,):
        return len(self._input_ids)

    def __getitem__(self, key):
        match key:
            case "input_ids":
                return self._input_ids
            case "attention_mask":
                return self._attention_mask
            case "labels":
                return self._labels
            case _:
                raise KeyError(f"Key {key} not found")

    def __setitem__(self, key, value):
        match key:
            case "input_ids":
                self._input_ids = value
            case "attention_mask":
                self._attention_mask = value
            case "labels":
                self._labels = value
            case _:
                raise KeyError(f"Key {key} not found")

    def __iter__(self):
        return iter(self.__dict__.keys())

    def to_dict(self):
        return {
            "input_ids": self["input_ids"],
            "attention_mask": self["attention_mask"],
            "labels": self["labels"]
        }
    def items(self):
        return self.to_dict().items()

    def keys(self):
        return self.to_dict().keys()

    def values(self):
        return self.to_dict().values()

    @property
    def input_ids(self):
        return self._input_ids

    @input_ids.setter
    def input_ids(self, value):
        self._input_ids = value

    @property
    def attention_mask(self):
        return self._attention_mask

    @attention_mask.setter
    def attention_mask(self, value):
        self._attention_mask = value

    @property
    def labels(self):
        return self._labels

    @labels.setter
    def labels(self, value):
        self._labels = value



class SumDataCallator(DataCollatorForLanguageModeling):
    def __init__(self, tokenizer, max_length,):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    def _tokenizing(self, text):
        return self.tokenizer(text,
                              truncation=True,
                              padding="max_length",
                              max_length=self.max_length,
                              return_tensors="pt")

    def __call__(self, batch):
        input_text = []
        labels = []
        for b in batch:
            input_text += [b["input_ids"]]
            labels += [b["labels"]]
        input_tokens = self._tokenizing(input_text)
        label_tokens = self._tokenizing(labels)

        return CallatorOutput(**{
            "input_ids": input_tokens['input_ids'].to(self.device),
            "attention_mask": input_tokens['attention_mask'].to(self.device),
            "labels": label_tokens['input_ids'].to(self.device),
        })
        # raise Exception("STOP")



Overwriting finetuning_datafunctions.py


#4. Train

In [46]:
#@title Start Training
#@markdown transformers trainer 이용, 추후 lightning 으로 이전 가능
%%writefile train.py
import nltk
import numpy as np
import torch
from torch.utils.data import DataLoader
from transformers import TrainingArguments, Trainer, TrainerCallback
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq, DataCollatorWithPadding, DataCollatorForLanguageModeling
from trl import SFTTrainer, SFTConfig
from ignite.metrics import Rouge

from peft_model import peft_model, tokenizer, lora_config
from finetuning_datasets import dataset, metric, max_source_length
from finetuning_datafunctions import SumDataCallator, formatting, preprocess_function


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
peft_model.to(device)

class ColabTrainer(SFTTrainer):
    pass


# Callback Class
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, num_steps=500):
        self.num_steps = num_steps

    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step >= self.num_steps:
            control.should_training_stop = True

        return control

# metric function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


# data_collator = DataCollatorWithPadding(
#     tokenizer=tokenizer,
#     padding=True,
#     max_length=max_source_length,
#     return_tensors="pt")
data_collator = SumDataCallator(tokenizer, max_length=max_source_length)

training_args = TrainingArguments(
    output_dir="llm_output",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    num_train_epochs=4,
    weight_decay=0.01,
    # eval_strategy="epoch",
    save_strategy="no",
    # use_cpu=True,
    # load_best_model_at_end=True,
    remove_unused_columns=False,
    push_to_hub=True,
    logging_steps=1000,
    save_steps=1000,
    warmup_steps=0.03,
    gradient_accumulation_steps=4,
    fp16=True,
    save_total_limit=3,
    # logging_dir="llm_output/logs",
    optim="paged_adamw_8bit",
    report_to="tensorboard",
)

trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback()],
    peft_config=lora_config,
    # formatting_func=formatting,
)

trainer.train()


Overwriting train.py


In [48]:
#@title deepspeed config yaml file saving
#@markdown deep speed config for transformers trainer
%%writefile deepspeed_config.yaml
compute_environment: LOCAL_MACHINE
debug: false
deepspeed_config:
  deepspeed_multinode_launcher: standard
  gradient_accumulation_steps: 4
  offload_optimizer_device: none
  offload_param_device: none
  zero3_init_flag: true
  zero3_save_16bit_model: true
  zero_stage: 2
distributed_type: DEEPSPEED
downcast_bf16: 'yes'
machine_rank: 0
main_training_function: main
mixed_precision: bf16
num_machines: 1
num_processes: 1
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false

Overwriting deepspeed_config.yaml


In [49]:
%%shell
#@title Huggingface Deepspeed Trainer
#@markdown env variables and accelerate trainer
export CUDA_VISIBLE_DEVICES=0
export CUDA_LAUNCH_BLOCKING=1
export TORCH_USE_CUDA_DSA=0
export HF_DATASETS_CACHE='/content/hf_cache/'
accelerate launch --config_file "deepspeed_config.yaml" train.py
# python train.py

[2024-07-24 05:46:43,715] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible
2024-07-24 05:46:50.203912: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 05

CalledProcessError: Command '#@title Huggingface Deepspeed Trainer
#@markdown env variables and accelerate trainer
export CUDA_VISIBLE_DEVICES=0
export CUDA_LAUNCH_BLOCKING=1
export TORCH_USE_CUDA_DSA=0
export HF_DATASETS_CACHE='/content/hf_cache/'
accelerate launch --config_file "deepspeed_config.yaml" train.py
# python train.py
' returned non-zero exit status 1.

## Training code to Lightning module

In [25]:
%%writefile l_datamodule.py
#@title Lightning Data Moudle
#@markdown Lightning Data Loading Modules

import lightning as L
import torch
from torch.utils.data import DataLoader

from transformers import DataCollatorForSeq2Seq
from peft_model import peft_model, tokenizer
from finetuning_datasets import dataset,  max_source_length
from finetuning_datafunctions import SumDataCallator


class FTDataModule(L.LightningDataModule):
    def __init__(self, train_dataset, val_dataset, test_dataset, data_collator, train_batch_size, eval_batch_size, training_args):
        super().__init__()
        self.train_dataset = dataset["train"]
        self.val_dataset = dataset["validation"]
        self.test_dataset = dataset["test"]
        self.data_collator = SumDataCallator(tokenizer, max_length=max_source_length,)
        self.train_batch_size = train_batch_size
        self.eval_batch_size = eval_batch_size
        self.training_args = training_args

    def _get_dataloader(self, dataset, eval_mode: bool = False):
        return DataLoader(dataset=dataset,
                          batch_size=self.train_batch_size if eval_mode else self.eval_batch_size,
                          shuffle=not eval_mode,
                          num_workers=8,
                          collate_fn=self.data_collator)

    def train_dataloader(self):
        return self._get_dataloader(dataset=self.train_dataset)

    def val_dataloader(self):
        return self._get_dataloader(dataset=self.val_dataset, eval_mode=True)

    def test_dataloader(self):
        return self._get_dataloader(dataset=self.test_dataset, eval_mode=True)


Writing l_datamodule.py


In [26]:
%%writefile l_model.py
#@title Lightning Model
#@markdown Lightning Modules and Training step

import lightning as L
import torch

from transformers import DataCollatorForSeq2Seq
from peft_model import peft_model, tokenizer
from bitsandbytes.optim import AdamW, Lion
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
from finetuning_datasets import dataset
from torchmetrics.functional.text.rouge import rouge_score


class LLamaFTLightningModule(L.LightningModule):
    def __init__(self, data_collator, learning_rate: float = 2e-5):
        super().__init__()
        self.save_hyperparameters()
        self.model = peft_model
        self.tokenizer = tokenizer
        # self.data_collator = DataCollatorForSeq2Seq(tokenizer, model=peft_model)
        self.learning_rate = learning_rate

    def _get_rouge_score(self, predictions, labels):
        generated_tokens = predictions.argmax(dim=-1)
        decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = self.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        return rouge_score(preds=decoded_preds, target=decoded_labels)

    def _log(self, log_name, value, batch_size):
        self.log(
            log_name,
            value if value.device == self.model.device else value.to(self.model.device),
            prog_bar=True,
            on_step=True,
            on_epoch=True,
            batch_size=batch_size,
            sync_dist=True)

    def _batch_device_correction(self, batch):
        for k, v in batch.items():
            if v.device != self.model.device:
                batch[k] = v.to(self.model.device)
        return batch

    def forward(self, input_ids, attention_mask, labels):
        # print(input_ids.shape, input_ids.min(), input_ids.max())
        return self.model(**{
            "input_ids":input_ids,
            "attention_mask":attention_mask,
            "labels": labels
            })

    def training_step(self, batch, batch_idx):
        batch = self._batch_device_correction(batch)
        outputs = self(input_ids=batch.input_ids,
                       attention_mask=batch.attention_mask,
                       labels=batch.labels)
        rouge_score = self._get_rouge_score(outputs.logits, batch.labels)
        loss = outputs.loss
        self._log("train_loss", loss, self.trainer.datamodule.train_batch_size,)
        for k, v in rouge_score.items():
            self._log(f"train_{k}", v, self.trainer.datamodule.train_batch_size,)

        return loss

    def validation_step(self, batch, batch_idx):
        batch = self._batch_device_correction(batch)
        outputs = self(input_ids=batch.input_ids,
                       attention_mask=batch.attention_mask,
                       labels=batch.labels)
        rouge_score = self._get_rouge_score(outputs.logits, batch.labels)
        val_loss = outputs.loss
        self._log("val_loss", val_loss, self.trainer.datamodule.eval_batch_size,)
        for k, v in rouge_score.items():
            self._log(f"val_{k}", v, self.trainer.datamodule.eval_batch_size,)

    def configure_optimizers(self):
        optimizer = Lion(params=self.model.parameters(),
                         lr=self.learning_rate,
                         weight_decay=0.01,
                         optim_bits=32,)
        scheduler = CosineAnnealingWarmRestarts(optimizer,
                                                T_0=10,
                                                T_mult=2,
                                                eta_min=0.00001)
        # scheduler = ReduceLROnPlateau(optimizer=optimizer, mode="min")
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
                "interval": "step",
                "frequency": 1,

            },
        }




Writing l_model.py


In [27]:
%%writefile l_trainer.py
#@title Trainer
#@markdown Lightning cli trainer

import os
import lightning as L
from lightning.pytorch.cli import LightningCLI, LightningArgumentParser
from lightning.pytorch.strategies.deepspeed import DeepSpeedStrategy
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from transformers import DataCollatorForSeq2Seq

from l_datamodule import FTDataModule
from l_model import LLamaFTLightningModule
from peft_model import peft_model, tokenizer
from finetuning_datasets import dataset

os.environ["TOKENIZERS_PARALLELISM"] = "0"


if __name__ == "__main__":
    L.pytorch.cli_lightning_logo()
    training_args = LightningArgumentParser()
    cli = LightningCLI(
        model_class=LLamaFTLightningModule,
        datamodule_class=FTDataModule,
        seed_everything_default=42,
        trainer_defaults={
            "reload_dataloaders_every_n_epochs": 1,
            "strategy": "deepspeed",
            "precision": "bf16-mixed",
            "accumulate_grad_batches": 4,
            "profiler": "PassThroughProfiler",
            "logger": [WandbLogger(project="LLM-Finetuning"),],
            "callbacks": [EarlyStopping(monitor="val_loss", patience=5), LearningRateMonitor()]
        },
        save_config_callback=None)
    # cli.add_arguments_to_parser(training_args)


Writing l_trainer.py


In [28]:
#@title Start Training
#@markdown 실험 결과
#@markdown
#@markdown * batch_size <b>2</b> 넘기는 경우 OOM
#@markdown * DeepSpeed의 경우 GPU Ram 20GB로 7B finetuning 가능
#@markdown * DeepSpeed의 경우, 7B L4 GPU에서 사용 가능
#@markdown * 70B의 경우 RAM에서 Weight 가져오다 OOM
#@markdown * 1.5B T4 GPU에서 성공

%%shell
export CUDA_VISIBLE_DEVICES=0
export CUDA_LAUNCH_BLOCKING=1


python l_trainer.py fit \
    --trainer.max_epochs 4 \
    --model.learning_rate 5e-5 \
    --data.train_batch_size 3 \
    --data.eval_batch_size 1

#    --trainer.fast_dev_run 1\

2024-07-24 04:19:55.864733: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 04:19:55.864804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 04:19:55.866142: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 04:19:57.113219: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, u

CalledProcessError: Command 'export CUDA_VISIBLE_DEVICES=0
export CUDA_LAUNCH_BLOCKING=1


python l_trainer.py fit \
    --trainer.max_epochs 4 \
    --model.learning_rate 5e-5 \
    --data.train_batch_size 3 \
    --data.eval_batch_size 1

#    --trainer.fast_dev_run 1\
' returned non-zero exit status 1.

# Model Saving

In [ ]:
#@title ONNX model save
#@markdown ONNX 로 모델 변형 후 저장
from optimum.onnxruntime import ORTModelForSequenceClassification, ORTModelForCausalLM

model_checkpoint = "./" #@param{"type":"string"}
save_directory = "./" #@param{"type":"string"}

ort_model = ORTModelForCausalLM.from_pretrained(model_checkpoint, export=True)
ort_model.save_pretrained(save_directory)